## **Installing pre-reqs**

In [1]:
!pip -q install faiss-cpu sentence-transformers langchain langchain-community langchain-text-splitters pypdf python-docx llama-cpp-python pandas tqdm requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1

## **Setting up google drive and folders**

In [2]:
from google.colab import drive
drive.mount('/content/drive')  # uncomment if you want persistence

import os, pathlib
BASE_DIR = "/content/drive/MyDrive/Colab"
DATA_DIR = f"{BASE_DIR}/dataset_raw"   # raw zips/files. Add the BRD here!
DOCS_DIR = f"{BASE_DIR}/docs"          # normalized text
OUT_DIR  = f"{BASE_DIR}/rag_outputs"   # indices, exports, model
for d in (DATA_DIR, DOCS_DIR, OUT_DIR): pathlib.Path(d).mkdir(parents=True, exist_ok=True)

Mounted at /content/drive


## **Normalize everything to plain text**

In [3]:
import re, glob
from pypdf import PdfReader
from docx import Document

def file_to_text(path):
    p = path.lower()
    if p.endswith(".pdf"):
        try:
            pages = []
            reader = PdfReader(path)
            for i, pg in enumerate(reader.pages, start=1):
                t = pg.extract_text() or ""
                pages.append(f"[PAGE {i}] {t}")
            return "\n".join(pages)
        except: return ""
    if p.endswith(".docx"):
        try:
            doc = Document(path)
            return "\n".join(p.text for p in doc.paragraphs)
        except: return ""
    if any(p.endswith(ext) for ext in (".txt",".md",".rst",".csv")):
        try:
            with open(path, "r", errors="ignore") as f: return f.read()
        except: return ""
    return ""  # unsupported

def clean_text(s):
    s = s.replace("\x00"," ")
    s = re.sub(r"[ \t]+"," ", s)
    s = re.sub(r"\n{3,}","\n\n", s)
    return s.strip()

import os, pathlib
count=0
for root, _, files in os.walk(DATA_DIR):
    for fn in files:
        src = os.path.join(root, fn)
        txt = clean_text(file_to_text(src))
        if txt and len(txt) > 50:
            out = f"{DOCS_DIR}/{pathlib.Path(fn).stem}.txt"
            with open(out, "w") as f: f.write(txt)
            count += 1
print("Normalized text files:", count)

Normalized text files: 1


## **Chunking, Embedding, FAISS**

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, pickle, glob, os

# Load normalized docs
docs = []
for fp in sorted(glob.glob(f"{DOCS_DIR}/*.txt")):
    with open(fp, "r", errors="ignore") as f:
        docs.append({"source": os.path.basename(fp), "text": f.read()})
print("Docs:", len(docs))

# Chunk
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=50,
    separators=["\n## ","\n# ","\n\n","\n"," ",""]
)
chunks = []
for d in docs:
    for ch in splitter.split_text(d["text"]):
        chunks.append({"source": d["source"], "text": ch})
print("Chunks:", len(chunks))

# Embeddings + FAISS (cosine via normalized IP)
EMB_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(EMB_NAME)

B=128; vecs=[]
texts = [c["text"] for c in chunks]
for i in range(0, len(texts), B):
    vecs.append(embed_model.encode(texts[i:i+B], convert_to_numpy=True, normalize_embeddings=True))
vecs = np.vstack(vecs).astype("float32")

index = faiss.IndexFlatIP(vecs.shape[1])
index.add(vecs)

# Persist
with open(f"{OUT_DIR}/chunks.pkl","wb") as f: pickle.dump(chunks,f)
faiss.write_index(index, f"{OUT_DIR}/faiss.index")
with open(f"{OUT_DIR}/emb_model.txt","w") as f: f.write(EMB_NAME)
print("Index built.")

Docs: 1
Chunks: 26


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Index built.


**Retriever helper**

In [5]:
def retrieve(query, k=8):
    q = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
    D, I = index.search(q, k)
    hits=[]
    for rank,(idx,score) in enumerate(zip(I[0],D[0]), start=1):
        hits.append({"rank":rank,"score":float(score),"source":chunks[idx]["source"],"text":chunks[idx]["text"]})
    return hits

## **Local LLM via llama_cpp**


In [6]:
import os, subprocess, pathlib

MODEL_URL = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
MODEL_PATH = f"{OUT_DIR}/mistral-7b-instruct.Q4_K_M.gguf"
if not os.path.exists(MODEL_PATH):
    !wget -q -O "$MODEL_PATH" "$MODEL_URL"

from llama_cpp import Llama
llm = Llama(model_path=MODEL_PATH, n_ctx=8192, n_threads=8, n_gpu_layers=2, temperature=0.2)

def chat_local(messages, max_tokens=1024, temperature=0.2):
    out = llm.create_chat_completion(messages=messages, temperature=temperature, max_tokens=max_tokens)
    return out["choices"][0]["message"]["content"]

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/drive/MyDrive/Colab/rag_outputs/mistral-7b-instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model

## **Prompt and RAG generation**

In [12]:
#Below is the structured, detailed prompt we'll be sending to the LLM. We can modify it according to our needs.
TESTCASE_SYSTEM = """You are a senior QA engineer. Generate rigorous, unambiguous test cases
based ONLY on the provided context snippets. If context is missing details, list assumptions and gaps."""

TESTCASE_USER_TEMPLATE = """Context (RAG snippets):
---
{context}
---
Task: Generate test cases for the scope: "{scope}".
If acceptance criteria (AC) appear in the context, map each test to at least one AC id/phrase.

Output (Markdown):
- Feature: <name>
- Scope: <scope>
- Assumptions/Notes: <gaps or clarifications needed>
- Test Cases (table):
  | ID | Title | Type (pos/neg/edge) | Pre-Conditions | Steps | Expected Result | Priority (H/M/L) | MapsTo (AC id/phrase) |
After the table:
- Coverage: bullets of ACs covered and any missing ACs - give details about them - what are they?
- Additional Negative/Edge Ideas: bullets
"""

def make_context(hits, max_chars=1000):
    blocks=[]
    for h in hits:
        t = h["text"].strip()
        if len(t) > max_chars: t = t[:max_chars] + "..."
        blocks.append(f"[{h['source']} • score={h['score']:.2f}]\n{t}")
    return "\n\n".join(blocks)

def generate_test_cases(scope_query, k=8, use="local"):
    hits = retrieve(scope_query, k=k)
    ctx = make_context(hits)
    messages = [
        {"role":"system","content":TESTCASE_SYSTEM},
        {"role":"user","content":TESTCASE_USER_TEMPLATE.format(context=ctx, scope=scope_query)}
    ]
    out = chat_local(messages)  # or chat_api(messages)
    return out, hits

## Defining function to save model outputs to drive.

In [13]:
import os, time, re
import pandas as pd

def parse_markdown_table(md_text: str):
    """
    Extracts the first Markdown table from LLM output and returns as DataFrame.
    Assumes '|' separated table rows with a header line.
    """
    lines = [ln.strip() for ln in md_text.splitlines() if ln.strip().startswith("|")]
    if not lines:
        return pd.DataFrame()

    # Remove markdown alignment row (| --- | --- | etc.)
    table_lines = [ln for ln in lines if not re.match(r'^\|\s*:?-+:?\s*\|', ln)]
    rows = [[c.strip() for c in ln.strip("|").split("|")] for ln in table_lines]

    if len(rows) < 2:
        return pd.DataFrame()

    header, body = rows[0], rows[1:]
    return pd.DataFrame(body, columns=header)

def generate_and_save(scope_query, k=8, use="local"):
    # Run your existing generator
    md_text, evidence = generate_test_cases(scope_query, k=k, use=use)

    # Timestamped filenames
    ts = time.strftime("%Y%m%d-%H%M%S")
    base = scope_query.strip().replace(" ", "_")[:50]

    md_path = os.path.join(OUT_DIR, f"{base}_{ts}.md")
    ev_path = os.path.join(OUT_DIR, f"{base}_{ts}_evidence.txt")
    csv_path = os.path.join(OUT_DIR, f"{base}_{ts}.csv")

    # Save markdown
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(md_text)

    # Save evidence
    with open(ev_path, "w", encoding="utf-8") as f:
        for h in evidence:
            f.write(f"[{h['source']} • score={h['score']:.2f}]\n{h['text']}\n\n")

    # Parse table → CSV
    df = parse_markdown_table(md_text)
    if not df.empty:
        df.to_csv(csv_path, index=False)
        print(f"✅ Saved CSV with {len(df)} rows: {csv_path}")
    else:
        print("⚠️ No markdown table parsed, skipping CSV export.")

    print(f"✅ Saved outputs:\n- {md_path}\n- {ev_path}")
    return md_text, evidence

In [15]:
#trying it
md_text, evidence = generate_test_cases(
    "As a ServiceNow administrator, I want to replicate Incident, Change, CMDB, and User tables into Oracle, so that reporting teams can query them directly in Oracle.",
    k=10, use="local"
)
print(md_text)

Llama.generate: 35 prefix-match hit, remaining 834 prompt tokens to eval
llama_perf_context_print:        load time =  264787.83 ms
llama_perf_context_print: prompt eval time =  250857.62 ms /   834 tokens (  300.79 ms per token,     3.32 tokens per second)
llama_perf_context_print:        eval time =  636877.51 ms /  1023 runs   (  622.56 ms per token,     1.61 tokens per second)
llama_perf_context_print:       total time =  889394.95 ms /  1857 tokens
llama_perf_context_print:    graphs reused =        990


 - Feature: ServiceNow to Oracle Data Replication
- Scope: Replication of Incident, Change, CMDB, and User tables from ServiceNow to Oracle
- Assumptions/Notes:
  - ServiceNow API credentials and permissions are available and properly configured.
  - Oracle DB access and schema preparation are completed.
  - IT Security approvals have been granted.
  - ServiceNow API rate limits will be respected.
  - Only active records will be replicated in incremental mode.
  - Network connectivity between ServiceNow and Oracle is available.

- Test Cases (table):
  | ID | Title | Type (pos/neg/edge) | Pre-Conditions | Steps | Expected Result | Priority (H/M/L) | MapsTo (AC id/phrase) |
  | 1 | Successful replication of Incident table | Positive | ServiceNow API credentials and permissions, Oracle DB access, and schema prepared | Configure and schedule replication job for Incident table in ServiceNow, wait for the job to complete, check Oracle for the replicated data | Incident table is fully replic

In [11]:
# print(evidence)

[{'rank': 1, 'score': 0.636603593826294, 'source': 'ServiceNow_to_Oracle_BRD_Full.txt', 'text': '2. Executive Summary\nThe purpose of this application is to seamlessly replicate ServiceNow table data into an Oracle'}, {'rank': 2, 'score': 0.5826738476753235, 'source': 'ServiceNow_to_Oracle_BRD_Full.txt', 'text': 'secure, and near real-time duplication of ServiceNow data into Oracle.\n3. Business Objectives\n1. Automate data replication between ServiceNow and Oracle.'}, {'rank': 3, 'score': 0.5514274835586548, 'source': 'ServiceNow_to_Oracle_BRD_Full.txt', 'text': '2. Ensure accuracy, consistency, and integrity of ServiceNow data in Oracle.\n3. Enable Oracle-native reporting and advanced analytics on ServiceNow operational data.'}, {'rank': 4, 'score': 0.536774754524231, 'source': 'ServiceNow_to_Oracle_BRD_Full.txt', 'text': '10. Acceptance Criteria\n- AC-1: User can configure and schedule replication jobs.\n- AC-2: ServiceNow tables selected are fully replicated into Oracle.'}, {'rank'

In [14]:
#trying it
md_text, evidence = generate_and_save(
    "As a business owner, I want to configure replication schedules (daily, hourly), so that data freshness matches business needs.",
    k=10, use="local"
)
print(md_text)

Llama.generate: 35 prefix-match hit, remaining 879 prompt tokens to eval
llama_perf_context_print:        load time =  264787.83 ms
llama_perf_context_print: prompt eval time =  260465.20 ms /   879 tokens (  296.32 ms per token,     3.37 tokens per second)
llama_perf_context_print:        eval time =  368026.86 ms /   590 runs   (  623.77 ms per token,     1.60 tokens per second)
llama_perf_context_print:       total time =  629168.86 ms /  1469 tokens
llama_perf_context_print:    graphs reused =        571


✅ Saved CSV with 5 rows: /content/drive/MyDrive/Colab/rag_outputs/As_a_business_owner,_I_want_to_configure_replicati_20250925-200341.csv
✅ Saved outputs:
- /content/drive/MyDrive/Colab/rag_outputs/As_a_business_owner,_I_want_to_configure_replicati_20250925-200341.md
- /content/drive/MyDrive/Colab/rag_outputs/As_a_business_owner,_I_want_to_configure_replicati_20250925-200341_evidence.txt
 - Feature: Data Replication Scheduling
- Scope: Configure replication schedules (daily, hourly)
- Assumptions/Notes:
  - ServiceNow API and Oracle DB are accessible
  - Replication jobs are already configured

- Test Cases (table):

| ID | Title | Type (pos/neg/edge) | Pre-Conditions | Steps | Expected Result | Priority (H/M/L) | MapsTo (AC id/phrase) |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | Configure daily replication schedule | Positive | Replication jobs are created | Set up a daily schedule in the replication configuration | Data is replicated daily as scheduled | High | AC-1, AC-2

In [ ]:
# Other Sample User Stories (Prompts)

# Data Replication
# 	•	US-1: As a ServiceNow administrator, I want to replicate Incident, Change, CMDB, and User tables into Oracle, so that reporting teams can query them directly in Oracle.
# 	•	US-2: As a data analyst, I want ServiceNow tables to update incrementally in Oracle, so that I can always access fresh data without reloading entire tables.

# Error Handling & Logging
# 	•	US-3: As a QA engineer, I want failed sync jobs to log detailed error messages and retry automatically, so that I can track failures and recover without manual intervention.
# 	•	US-4: As a system administrator, I want email alerts when a sync fails, so that I can quickly take corrective action.

# Scheduling
# 	•	US-5: As a business owner, I want to configure replication schedules (daily, hourly), so that data freshness matches business needs.

# Security & Compliance
# 	•	US-6: As a security officer, I want data transfers to be encrypted (TLS 1.2+), so that compliance with security standards is ensured.
# 	•	US-7: As a compliance auditor, I want replication logs to be retained for 90 days, so that audit trails are available when required.

# Performance & Scale
# 	•	US-8: As a DBA, I want the replication process to handle 1M records within 2 hours, so that large ServiceNow tables can be synchronized overnight without delays.
# 	•	US-9: As a ServiceNow admin, I want the tool to support adding new tables without code changes, so that the solution scales easily with future needs.